# Feature 데이터에 모델 점수를 연결

In [ ]:
import pandas as pd
import numpy as np
import pickle
from pathlib import Path
from tqdm import tqdm

base_path = Path('./experiment/matches_scores')
csv_path = Path('./Dataset/matches_csv')
save_path = Path('./experiment/feature_scores.csv')
models = [model.name for model in list(base_path.glob('*'))]
match_list = [match.name[:-4] for match in list((base_path / models[0]).glob('*'))]

first = True
for match in tqdm(match_list):
    features = pd.read_csv((csv_path / f'{match}.csv'))
    features.sort_values(['player', 'time'], ascending=[True, False]).reset_index(drop=True)
    features.drop(['player_level', 'skill_level', 'tower_diff', 'is_valid'], axis='columns', inplace=True)
    features.insert(0, 'match_no', match)
    for model in models:
        features[model] = np.nan
        reverse = True if 'reverse' in model else False
        score_file = base_path / model / f'{match}.pkl'
        with score_file.open('rb') as file: scores = pickle.load(file)
        for player, p_scores in enumerate(scores):
            p_scores = p_scores.squeeze().tolist()
            if reverse and isinstance(p_scores, list): p_scores.reverse()
            features.loc[features['player']==(player+1), model] = p_scores
    if first:
        features.to_csv(save_path, header=True, index=False, mode='w')
        first = False
    else: features.to_csv(save_path, header=False, index=False, mode='a')